In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import os
import numpy as np
import pandas as pd
import re
from nltk.stem.porter import * 
from wordcloud import WordCloud
import spacy
%matplotlib inline


# storing and analysis
import numpy as np
import pandas as pd
import re

# visualization
import matplotlib.pyplot as plt
import warnings
import nltk
import string
import seaborn as sns

#import text classification modules
import os
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from nltk.stem.porter import * 
from wordcloud import WordCloud
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
os.listdir('../input/edsa-climate-change-belief-analysis-2021')

In [9]:
import pandas as pd
train_df = pd.read_csv('../input/edsa-climate-change-belief-analysis-2021/train.csv')
test_df = pd.read_csv('../input/edsa-climate-change-belief-analysis-2021/test.csv')
train_df.head()



In [10]:
test_df.head()

In [11]:
#Print out the Shape of the training data and the testing data
print(train_df.shape)
print(test_df.shape)

In [12]:
#Confirm if we have any class inbalance
train_df['sentiment'].value_counts()

In [13]:
#Confirm if the dataset have any null values
train_df.isnull().sum()

In [14]:
#Combining both train and test data set before data cleaning 
data = train_df.append(test_df, ignore_index=True)

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt


In [ ]:
#remove twitter handle and converting to lower case

data['tidy_message'] = np.vectorize(remove_pattern)(data['message'], "@[\w]*")
data['tidy_message'] = data['tidy_message'].str.replace('RT :',' ')
data['tidy_message'] = data['tidy_message'].apply(lambda x: x.lower())
data.head()

In [ ]:
#Removing short words  - can extract more meaning out of data

short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great",
"8yo":"eight year old"
}

In [ ]:
def lookup_dict(text, dictionary):
    for word in text.split(): #split the text into a list and loop through the list for find the words
        if word.lower() in dictionary: #Lower the words and see if they are in the dictionary
            if word.lower() in text.split(): #lower the words and see if they are in the text list
                text = text.replace(word, dictionary[word.lower()]) #replace the word in the text split with the values in the dictionary
    return text


#Perform lookup of short words and return the string with the full meaning
data['tidy_message'] = data['tidy_message'].apply(lambda x: lookup_dict(x,short_word_dict))

In [ ]:
contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:


#Perform lookup of apostrophe words and return the string with the full meaning
data['tidy_message'] = data['tidy_message'].apply(lambda x: lookup_dict(x,contractions))# Use the contraction dictionary to replace the aprostophe words in data with the full meaning
data.head()



In [ ]:
# Emoji Lookup

tweets_text = data.tidy_message.str.cat() #Concatenate the tweet string and assign the variable 'tweets_text'
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text)) #Create a regular expression that finds the emojis in the tweets
emos_count = [] #Initialise an empty list 'emo_count'
for emo in emos: #Loop through emoji
    emos_count.append((tweets_text.count(emo), emo)) #Add emojis found into the emo_count list
sorted(emos_count,reverse=True) #Sort the list in reverse order


In [ ]:
emoticon_dict={
'XD':'happy',
';)':'happy',
':-)':'happy',
';-)':'happy',
':P':'happy', 
':)':'happy',
'x ':'happy',
':(':'sad',
':/':'sad'
}



#Lookup the emojis in the tweets and replace them with the meaning behind the tweet
data['tidy_message'] = data['tidy_message'].apply(lambda x: lookup_dict(x,emoticon_dict))# Use emoticon_dict to replace decode emojis in the dataset



In [ ]:
#Function to remove the remaining emojis in the tweets
def remove_emoji(message):
  
    emoji_pattern = re.compile("["                   # Create a regular expression
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', message)

data['tidy_message'] = data['tidy_message'].apply(lambda x: remove_emoji(x))

In [ ]:
data['tidy_message'] = data['tidy_message'].str.replace("[^a-zA-Z#]", " ")
data.head(10)

In [ ]:
# # remove special characters, numbers, punctuations
data['tidy_message'] = data['tidy_message'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data.head()

In [ ]:
#Tokenization

tokenized_tweet = data['tidy_message'].apply(lambda x: x.split()) #Tokenize the dataset
tokenized_tweet.head()



In [ ]:
#Stemming


#Use PorterStemmer() to strip suffixes from the words
from nltk.stem.porter import * #Import * from nltk.stem.porter
stemmer = PorterStemmer() #Initialize an instance of PorterStemmer and assign it a variable 'stemmer'

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) #Stem the dataset
tokenized_tweet.head()



In [ ]:


#We will now bring the tokens back together
for i in range(len(tokenized_tweet)): #Loop through token list of stems
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i]) #Join the list into a string of characters

data['tidy_message'] = tokenized_tweet #Assign the string of words to the dataset in the tid_message column



In [ ]:

#Split the dataset back to the training set and the testing set
train = data[:len(train_df)] #Split to the lenght of original training set
test = data[len(train_df):]  #Split to the length of original testing set
print(train.shape)
print(train['sentiment'].value_counts())

In [ ]:
#create a new length value column that contains the lengths of the messages
train['message_length'] = train['message'].apply(len)



In [ ]:
positive_words =' '.join([text for text in data['tidy_message'][data['sentiment'] == 1]]) #Words in the positve class
negative_words = ' '.join([text for text in data['tidy_message'][data['sentiment'] == -1]]) #Words in negative class
normal_words =' '.join([text for text in data['tidy_message'][data['sentiment'] == 0]]) #Words in the neutral class
news_words =' '.join([text for text in data['tidy_message'][data['sentiment'] == 2]]) #Words in the news class

In [ ]:
def word_cloud(class_words):
    wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(class_words)
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title("Most Common positive words")
    plt.axis('off')
    return plt.show()

In [ ]:
#Create a function to collect hashtags
def hashtag_extract(x):
    hashtags = [] #Initialize an empty list
    for i in x:   #Loop over the words in the tweet
        ht = re.findall(r"#(\w+)", i) #Create a regular expression to get the hashtags in a tweet
        hashtags.append(ht) #Add all those hashtags to the empty hashtag list
    return hashtags


In [ ]:


# extracting hashtags from the news
HT_news = hashtag_extract(data['tidy_message'][data['sentiment'] == 2])
# extracting hashtags from positive sentiments
HT_positive = hashtag_extract(data['tidy_message'][data['sentiment'] == 1])
# extract hashtags from neutral sentiments
HT_normal = hashtag_extract(data['tidy_message'][data['sentiment'] == 0])
# extracting hashtags from negative sentiments
HT_negative = hashtag_extract(data['tidy_message'][data['sentiment'] == -1])

# unnesting list of all sentiments
HT_news = sum(HT_news,[])
HT_positive = sum(HT_positive,[])
HT_normal = sum(HT_normal,[])
HT_negative = sum(HT_negative,[])



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
X = train['tidy_message'] #X is the features of the cleaned tweets
y = train['sentiment']    #Y is the target variable which is the train sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) #Splitting train set into training and testing data
#Print out the shape of the training set and the testing set
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# import and call the TFidfVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer #Import TFidfVectorizer from sklearn
tfidf = TfidfVectorizer()

In [ ]:
#import CountVectorizer and call it
from sklearn.feature_extraction.text import CountVectorizer #Import CountVectorizer from sklearn

cf= CountVectorizer() 

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# create a pipeline and fit it with a Logistic Regression
from sklearn.linear_model import LogisticRegression #Import Logistic Regression from sklearn

model = LogisticRegression(multi_class='ovr') #Call the Logistic Regression model and assign it to the variable 'model'

clf = Pipeline([('tfidf', tfidf), ('clf', model)]) #Create a pipeline with the TF-IDF Vectorizer with the logistic model


clf.fit(X_train, y_train) #Fit the training data to the pipeline

y_pred= clf.predict(X_test) #Make predictions and assign the predictions to y_pred

print('accuracy %s' % accuracy_score(y_pred, y_test)) #Print the accuracy
print('f1_score %s' % metrics.f1_score(y_test,y_pred,average='weighted')) #Print the weighted f1 score
print(classification_report(y_test, y_pred)) #Classification

In [ ]:
from sklearn.svm import LinearSVC #Import LinearSVC from sklearn 

classifier = LinearSVC() #Call LinearSVC and assign the variable 'classifier'

clf = Pipeline([('tfidf', tfidf), ('clf', classifier)]) #Create a pipeline with the tdidf

clf.fit(X_train, y_train) #Fit the model
y_pred = clf.predict(X_test) #Make predictions and assign the variable 'y_pred'

print('accuracy %s' % accuracy_score(y_pred, y_test)) #Print the accuracy
print('f1_score %s' % metrics.f1_score(y_test,y_pred,average='weighted')) #Print the f1-score
print(classification_report(y_test, y_pred)) #Print the classification report

In [ ]:
data.to_csv('MagdaMostert001.csv',index=False)